<a href="https://www.kaggle.com/code/mastersniffer/openai-custom-env-reinforcement-learning-tf?scriptVersionId=129180683" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
%pip install tensorflow==2.3.0 -q
%pip install gym -q 
%pip install keras==2.11.0 -q
%pip install keras-rl2 -q

ERROR: Could not find a version that satisfies the requirement tensorflow==2.3.0 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.13.0rc0)
ERROR: No matching distribution found for tensorflow==2.3.0
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
onnx 1.13.1 requires protobuf<4,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
kfp 1.8.20 requires google-api-python-client<2,>=1.7.8, but you have google-api-python-client 2.86.0 which is inc

In [2]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [3]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        self.state = 38 + random.randint(-3,3)
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = 38 + random.randint(-3,3)
        # Reset shower time
        self.shower_length = 60 
        return self.state
    

In [4]:
env = ShowerEnv()

/opt/conda/lib/python3.10/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [5]:
episodes = 10
for episode in range(1,episodes+1):
    state = env.reset()
    done = False
    score = 0
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-18
Episode:2 Score:-4
Episode:3 Score:-48
Episode:4 Score:-44
Episode:5 Score:-16
Episode:6 Score:-22
Episode:7 Score:-58
Episode:8 Score:-8
Episode:9 Score:-6
Episode:10 Score:-60


# Using TF

In [6]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
states = env.observation_space.shape # Get shape
actions = env.action_space.n # Get number of usable parameters

In [8]:
def build_model(states,actions):
    model = Sequential()
    model.add(Dense(24,activation = "relu", input_shape=states))
    model.add(Dense(24,activation = "relu"))
    model.add(Dense(actions, activation = 'linear'))
    return model

In [9]:
try:
    del model
except Exception:
    pass

In [10]:
model = build_model(states, actions)

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 24)                48        
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
 dense_2 (Dense)             (None, 3)                 75        
                                                                 
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


# Building Keras RL

In [12]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory


In [13]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=900_000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                nb_actions=actions, nb_steps_warmup=100, target_model_update=1e-2)
    return dqn

In [14]:
# Need this step to avoid errors
try:
    del model
except Exception:
    pass

model = build_model(states, actions)

In [15]:
import tensorflow as tf
dqn = build_agent(model, actions)
dqn.compile(tf.keras.optimizers.legacy.Adam(learning_rate=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=900_000, visualize=False, verbose=1)

Training for 900000 steps ...
Interval 1 (0 steps performed)
   52/10000 [..............................] - ETA: 9s - reward: -1.0000   

/opt/conda/lib/python3.10/site-packages/keras/engine/training_v1.py:2357: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


10000/10000 [==============================] - 75s 7ms/step - reward: -0.5400
166 episodes - episode_reward: -32.313 [-60.000, 32.000] - loss: 0.871 - mae: 5.677 - mean_q: -7.241

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 78s 8ms/step - reward: -0.4536
167 episodes - episode_reward: -27.425 [-60.000, 32.000] - loss: 1.635 - mae: 8.994 - mean_q: -12.810

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 77s 8ms/step - reward: -0.4236
167 episodes - episode_reward: -25.317 [-60.000, 42.000] - loss: 2.084 - mae: 10.073 - mean_q: -14.438

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 80s 8ms/step - reward: -0.3838
166 episodes - episode_reward: -23.060 [-60.000, 42.000] - loss: 2.041 - mae: 9.972 - mean_q: -14.291

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 81s 8ms/step - reward: -0.3634
167 episodes - episode_reward: -21.760 [-60.000, 48.000] -

In [16]:
scores = dqn.test(env, nb_episodes=1000, visualize=False)

Testing for 1000 episodes ...
Episode 1: reward: 60.000, steps: 60
Episode 2: reward: 58.000, steps: 60
Episode 3: reward: 60.000, steps: 60
Episode 4: reward: 60.000, steps: 60
Episode 5: reward: 60.000, steps: 60
Episode 6: reward: 60.000, steps: 60
Episode 7: reward: 58.000, steps: 60
Episode 8: reward: 60.000, steps: 60
Episode 9: reward: 60.000, steps: 60
Episode 10: reward: 58.000, steps: 60
Episode 11: reward: 60.000, steps: 60
Episode 12: reward: 60.000, steps: 60
Episode 13: reward: 60.000, steps: 60
Episode 14: reward: 60.000, steps: 60
Episode 15: reward: 58.000, steps: 60
Episode 16: reward: 58.000, steps: 60
Episode 17: reward: 60.000, steps: 60
Episode 18: reward: 58.000, steps: 60
Episode 19: reward: 60.000, steps: 60
Episode 20: reward: 60.000, steps: 60
Episode 21: reward: 60.000, steps: 60
Episode 22: reward: 60.000, steps: 60
Episode 23: reward: 60.000, steps: 60
Episode 24: reward: 58.000, steps: 60
Episode 25: reward: 60.000, steps: 60
Episode 26: reward: 60.000, s

In [17]:
print(np.mean(scores.history['episode_reward']))

59.394
